In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install openmim
!mim install mmengine
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html


In [29]:
!git clone https://github.com/open-mmlab/mmaction2.git
!cd mmaction2
!pip install -r requirements.txt
!pip install -v -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
   

In [13]:
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 24.5 MB/s eta 0:00:00


In [22]:
!mkdir checkpoints

# Download TSN R50 model trained on Kinetics-400 RGB dataset
!wget https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth -P checkpoints/


mkdir: cannot create directory ‘checkpoints’: File exists
--2024-12-30 07:06:20--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 192.169.122.180, 192.169.122.181, 192.169.122.182, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|192.169.122.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579339 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth.2’

tsn_r50_1x1x3_100e_ 100%[===================>]  93.06M  68.5MB/s    in 1.4s    

2024-12-30 07:06:21 (68.5 MB/s) - ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth.2’ saved [97579339/97579339]



In [23]:
from mmengine import Config

# Load the updated TSN config file
config_file = 'mmaction2/configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py'
cfg = Config.fromfile(config_file)

# Update train dataset configuration
cfg.train_dataloader.dataset = dict(
    type='RawframeDataset',
    modality='RGB',
    ann_file='/content/drive/MyDrive/mmaction/annotations/train.txt',
    data_prefix={'img': '/content/drive/MyDrive/mmaction/videos/'},  # Corrected data_prefix to match actual file structure
    pipeline=[
        dict(type='SampleFrames', clip_len=5, frame_interval=1, num_clips=1),
        dict(type='RawFrameDecode', io_backend='disk'),
        dict(type='Resize', scale=(-1, 256)),
        dict(type='CenterCrop', crop_size=224),
        dict(type='FormatShape', input_format='NCHW'),  # Ensure channels-first format
        dict(type='PackActionInputs', meta_keys=['img_shape', 'label'])  # Package inputs
    ],
    test_mode=False
)

# Update validation dataset configuration
cfg.val_dataloader.dataset = dict(
    type='RawframeDataset',
    modality='RGB',
    ann_file='/content/drive/MyDrive/mmaction/annotations/val.txt',
    data_prefix={'img': '/content/drive/MyDrive/mmaction/videos/'},  # Corrected data_prefix to match actual file structure
    pipeline=[
        dict(type='SampleFrames', clip_len=5, frame_interval=1, num_clips=1),
        dict(type='RawFrameDecode', io_backend='disk'),
        dict(type='Resize', scale=(-1, 256)),
        dict(type='CenterCrop', crop_size=224),
        dict(type='FormatShape', input_format='NCHW'),  # Ensure channels-first format
        dict(type='PackActionInputs', meta_keys=['img_shape', 'label'])  # Package inputs
    ],
    test_mode=True
)

cfg.model.data_preprocessor = dict(
    type='ActionDataPreprocessor',
    mean=[123.675, 116.28, 103.53],  # RGB mean values
    std=[58.395, 57.12, 57.375],  # RGB std values
    format_shape='NCHW'  # Channels-first format
)


# Update model configuration
cfg.model.cls_head.num_classes = 2  # Strike and neutral classes
cfg.train_cfg.max_epochs = 20  # Set maximum epochs for training

# Update working directory
cfg.work_dir = '/content/mmaction_work_dir'

# Save the modified config
custom_config_path = '/content/mmaction2/tsn_custom_config.py'
cfg.dump(custom_config_path)
print(f"Custom configuration file saved to: {custom_config_path}")


Custom configuration file saved to: /content/mmaction2/tsn_custom_config.py


In [30]:
!python tools/train.py /content/mmaction2/tsn_custom_config.py --work-dir /content/mmaction_work_dir

12/30 07:09:31 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 926793465
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_

In [71]:
import os
import cv2
import torch
from pygifsicle import optimize
from mmaction.structures import ActionDataSample
from moviepy.editor import ImageSequenceClip
from torchvision.transforms import ToTensor
from mmaction.apis import init_recognizer, inference_recognizer

# Declare class names directly
class_names = {0: "neutral", 1: "strike"}


# Paths
video_path = "/content/drive/MyDrive/test_video.mp4"
gif_path = "/content/drive/MyDrive/test_punch.gif"
output_frames_dir = "/content/frames_tp"
os.makedirs(output_frames_dir, exist_ok=True)

# Parameters
sequence_length = 5
config_file = "/content/mmaction_work_dir/tsn_custom_config.py"  # Path to your MMAction2 config
model_checkpoint = '/content/mmaction_work_dir/epoch_20.pth'  # Replace with your model's checkpoint path
# config_file = "/content/drive/MyDrive/tsn_custom_config.py"  # Path to your MMAction2 config
# model_checkpoint = '/content/drive/MyDrive/epoch_20.pth'  # Replace with your model's checkpoint path


# Initialize the recognizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = init_recognizer(config_file, model_checkpoint, device=device)

# Extract first 2 seconds from the video using OpenCV
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = 6 * fps  # First 2 seconds
frame_idx = 60

frames = []
while cap.isOpened() and frame_idx < frame_count:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
    frame_idx += 1

cap.release()

# Save individual frames
frame_paths = []
for i, frame in enumerate(frames):
    frame_path = os.path.join(output_frames_dir, f"frame_{i:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_paths.append(frame_path)

# Prepare sequences of 5 frames
sequence_length = 5
sequences = [frames[i:i + sequence_length] for i in range(0, len(frames), sequence_length) if len(frames[i:i + sequence_length]) == sequence_length]

# Perform inference for each sequence
predicted_labels = []
for sequence in sequences:
    # Save the sequence temporarily as a video
    temp_video_path = "/content/temp_sequence.mp4"
    out = cv2.VideoWriter(temp_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (sequence[0].shape[1], sequence[0].shape[0]))
    for frame in sequence:
        out.write(frame)
    out.release()

    # Perform inference
    results = inference_recognizer(model, temp_video_path)

    # Extract the prediction scores
    scores = results.get('pred_score', None)
    if scores is None:
        raise ValueError("No prediction scores found in the result object.")

    if isinstance(scores, torch.Tensor):
        scores = scores.cpu().numpy()

    # Map the highest score index to class label
    top_pred_idx = scores.argmax()
    top_label = class_names[top_pred_idx]  # Use the declared class names
    predicted_labels.append(top_label)

# Overlay labels on frames
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (0, 255, 0)  # Green
thickness = 2

for i, sequence in enumerate(sequences):
    label = predicted_labels[i]
    for j, frame in enumerate(sequence):
        frame_idx = i * sequence_length + j
        text = f"Label: {label}"
        cv2.putText(frame, text, (10, 30), font, font_scale, font_color, thickness)
        frame_paths[frame_idx] = os.path.join(output_frames_dir, f"frame_{frame_idx:04d}_labeled.jpg")
        cv2.imwrite(frame_paths[frame_idx], frame)

# Resize frames to reduce resolution
resized_frames = [
    cv2.resize(cv2.imread(path), (0, 0), fx=0.5, fy=0.5) for path in frame_paths
]

# Create the GIF while keeping the original frame rate and full color
clip = ImageSequenceClip(resized_frames, fps=fps)
clip.write_gif(gif_path, fps=fps)

print(f"GIF saved at {gif_path}")

Loads checkpoint by local backend from path: /content/mmaction_work_dir/epoch_20.pth
MoviePy - Building file /content/drive/MyDrive/test_punch.gif with imageio.


GIF saved at /content/drive/MyDrive/test_punch.gif


In [72]:
# Optimize the GIF using pygifsicle
optimized_gif_path = "/content/drive/MyDrive/tp.gif"
os.system(f"gifsicle -O3 --lossy=80 -o {optimized_gif_path} {gif_path}")


print(f"Optimized GIF saved at {optimized_gif_path}")

Optimized GIF saved at /content/drive/MyDrive/tp.gif
